In [2]:
import pandas as pd
import re
from collections import Counter

In [3]:
df = pd.read_excel("data/newest_dataset_lxx_mt_dss.xlsx").fillna("")

In [4]:
pd.set_option("display.max_columns", False)
pd.set_option("display.max_rows", False)
pd.set_option("display.max_colwidth", False)

In [6]:
set(df.lex)

{'BW>[',
 'δυσμή',
 'ἐπάγω',
 'εἰμί',
 'εἰσάγω',
 'εἰσπορεύω',
 'εἰσφέρω',
 'εἰσφέρω',
 'εἰσάγω',
 'εἰσέρχομαι',
 'εἴσειμι',
 'παραγίγνομαι',
 'ποιέω',
 'πορεύω',
 'προσάγω ',
 'προσέρχομαι',
 'προσφέρω',
 'συνάπτω',
 'τίθημι',
 'φέρω',
 'ἀναφέρω',
 'ἄγω',
 'ἐξέρχομαι',
 'ἐπάγω',
 'ἔρχομαι',
 'ἥκω'}

In [10]:
# 1. Filter only MT rows
mt_only = df[df['scroll'].str.upper() == 'MT']

# 2. Count how many times each Hebrew verb lexeme occurs per stem
stem_verb_counts = (
    mt_only
    .groupby(['stem', 'lex'])
    .size()
    .reset_index(name='n')
    .sort_values(['stem', 'n'], ascending=[True, False])
)

print(stem_verb_counts)

  stem   lex   n
0  hif  BW>[  44
1  hof  BW>[  1 
2  qal  BW>[  80


In [27]:
# 1. Normalize and extract shared numeric ID
df2 = df.copy()
df2['scroll'] = df2['scroll'].astype(str).str.upper()
df2['base_id'] = df2['verb_id'].astype(str).str.extract(r'(\d+)')

# 2. Separate MT (for stem) and LXX (for Greek lex)
mt = df2[df2['scroll'] == 'MT'][['base_id', 'stem']].rename(columns={'stem': 'mt_stem'})
lxx = df2[df2['scroll'] == 'LXX'][['base_id', 'lex']].rename(columns={'lex': 'greek_lex'})

# 3. Merge them on base_id (link Hebrew stem to Greek lex)
merged = mt.merge(lxx, on='base_id', how='inner')

# 4. Count how many times each Greek lex is used for each Hebrew stem
counts = (
    merged.groupby(['mt_stem', 'greek_lex'])
    .size()
    .reset_index(name='n')
    .sort_values(['mt_stem', 'n'], ascending=[True, False])
)

print(counts)

   mt_stem     greek_lex   n
10  hif     φέρω          14
4   hif     εἰσάγω        7 
2   hif     εἰσφέρω       6 
11  hif     ἀναφέρω       3 
0   hif     ἐπάγω       2 
3   hif     εἰσφέρω       2 
1   hif     εἰμί          1 
5   hif     εἰσέρχομαι    1 
6   hif     προσάγω       1 
7   hif     προσφέρω      1 
8   hif     συνάπτω       1 
9   hif     τίθημι        1 
12  hif     ἄγω           1 
13  hif     ἐπάγω         1 
14  hof     εἰσάγω        1 
19  qal     εἰσέρχομαι    33
18  qal     εἰσπορεύω     12
21  qal     παραγίγνομαι  12
26  qal     ἔρχομαι       12
27  qal     ἥκω           3 
15  qal     δυσμή         2 
20  qal     εἴσειμι       2 
24  qal     φέρω          2 
25  qal     ἐξέρχομαι     2 
16  qal     εἰμί          1 
17  qal     εἰσάγω        1 
22  qal     πορεύω        1 
23  qal     προσέρχομαι   1 
